# 1.0 - IMPORTS

## 1.1 - LIBRARIES

In [ ]:
import pandas                   as pd
import numpy                    as np
import matplotlib.pyplot        as plt
import plotly.express           as px
import seaborn                  as sns
import re
import inflection
import umap.umap_               as umap
from sklearn.cluster            import KMeans, DBSCAN
from sklearn.metrics            import silhouette_score, silhouette_samples
from sklearn.preprocessing      import MinMaxScaler, StandardScaler
from sklearn.decomposition      import PCA
from sklearn.manifold           import TSNE
from yellowbrick.cluster        import KElbowVisualizer, SilhouetteVisualizer
from sklearn.ensemble           import RandomForestRegressor
from sklearn.mixture            import GaussianMixture
from scipy.cluster.hierarchy    import linkage, dendrogram, fcluster
import sqlite3
from sqlalchemy import create_engine



## 1.2 - LOAD DATA

In [ ]:
df_raw = pd.read_csv('dataset/data.csv', encoding='latin-1')

## 1.3 - HELPER FUNCTIONS

### 1.3.1 - NA VOLUMETRY

In [ ]:
def nan_volumetry(data):
    df_na = pd.DataFrame()
    df_na['attributes'] = data.columns
    df_na['volumetry'] = ''
    df_na['percentual'] = ''

    for i in df_na.index:
        volumetry = data[df_na.loc[i, 'attributes']].isna().sum()
        percentual = (volumetry / data['customer_id'].count()) * 100
        df_na.loc[i, 'volumetry'] = volumetry
        df_na.loc[i, 'percentual'] = f"{percentual:.2f}%"  

    return df_na

### 1.3.2 - PLOT DIMENSIONS

In [ ]:
sns.set(rc = {'figure.figsize':(13,5)})

# 2.0 - DATA DESCRIPTION

In [ ]:
df2 = df_raw.copy()

## 2.1 - DATA DIMENSION

In [ ]:
print('Total of unique features:\n')
print(df2.apply(lambda x: x.nunique()))
print('\nNumber of rows: {}'.format(df2.shape[0]))
print('Number of cols: {}'.format(df2.shape[1]))

## 2.2 - RENAME COLUMNS

In [ ]:
old_columns = df2.columns
snakecase = lambda x: inflection.underscore(x)
new_columns = list(map(snakecase, old_columns))
df2.columns = new_columns

## 2.3 - DATA TYPES

In [ ]:
df2.dtypes

## 2.4 - NAN VOLUMETRY

In [ ]:
df2.isna().sum()

In [ ]:
nan_volumetry(df2)

### 2.4.1 - FILL NA

In [ ]:
df_missing = df2.loc[df2['customer_id'].isna(),:]
df_full = df2.loc[~df2['customer_id'].isna(),:]

In [ ]:
# create reference
df_backup = pd.DataFrame( df_missing['invoice_no'].drop_duplicates() )
df_backup['customer_id'] = np.arange( 19000, 19000+len( df_backup ), 1)

# merge original with reference dataframe
df2 = pd.merge( df2, df_backup, on='invoice_no', how='left' )

# coalesce 
df2['customer_id'] = df2['customer_id_x'].combine_first( df2['customer_id_y'] )

# drop extra columns
df2 = df2.drop( columns=['customer_id_x', 'customer_id_y'], axis=1 )
df2.head(6)

In [ ]:
df2.isna().sum()

## 2.5 - CHANGE DATA TYPES

In [ ]:
df2['invoice_date'] = pd.to_datetime(df2['invoice_date'])
df2['customer_id'] = df2['customer_id'].astype(int)

In [ ]:
df2.dtypes

# 3.0 - DESCRIPTIVE STATISTICS

In [ ]:
df3 = df2.copy()
num_attributes = df3.select_dtypes(include = ['int64', 'float64'])
cat_attributes = df3.select_dtypes(include = 'object')

## 3.1 - NUMERICAL ATTRIBUTES

In [ ]:
# central tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T
             
# dispersion - desvio padrão, minimo, maximo, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( np.min ) ).T
d3 = pd.DataFrame( num_attributes.apply( np.max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# concatenate
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'mediana', 'std', 'skew', 'kurtosis']
m

### 3.1.1 - NUMERICAL ATTRIBUTES ANALYSIS

- Negative Quantity (Returning products?)
- Unity Price less than zero (Product discount?)

## 3.2 - CATEGORICAL ATTRIBUTES

In [ ]:
cat_attributes.columns

### INVOICE NUMBER

We have Invoice Number with letters and numbers, sendo assim iremos padronizar para uma variavel numerica pois se trata de uma ordem de compra.

In [ ]:
df_letter_invoices = df3.loc[df3['invoice_no'].apply( lambda x: bool( re.search( '[^0-9]+', x ) ) ), :]
df_letter_invoices.head()

print( 'Total number of invoices: {}'.format( len( df_letter_invoices ) ) )
print( 'Total number of negative quantity: {}'.format( len( df_letter_invoices[ df_letter_invoices['quantity'] < 0 ] ) ) )

Observando que a coluna Quantity indicam numeros negativos, podemos assumir que são dados relacionados a descontos, devolução ou estorno (cancelamento) de mercadorias pois correspondem a quantidade total de ordens com letras.

STOCK CODE

Analise de stocke code com apenas caracteres

In [ ]:
df3.loc[df3['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), :].sample(10)

In [ ]:
stock_code_letters = df3.loc[df3['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), 'stock_code'].unique()
stock_code_letters

### DESCRIPTION

We'll drop the Description column, because its not usefull or bring some insight ou metric to our analysis.

In [ ]:
df3.sample(5)

### COUNTRY

In [ ]:
print('Total of unique Countries: {}'.format(df3['country'].unique()))
print('\n Total of number of unique Countries: {}'.format(df3['country'].nunique()))
print('\n Percentual of orders per country:')
(df3['country'].value_counts(normalize = True)*100).apply(lambda x: '{:.2f}%'.format(x))

Quantity of Customers by Country

In [ ]:
df3[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending = False)

# 4.0 - FEATURE SELECTION

In [ ]:
df4 = df3.copy()

In [ ]:
# === Numerical attributes ====
df4 = df4.loc[df4['unit_price'] >= 0.04, :]

# === Categorical attributes ====
df4 = df4[~df4['stock_code'].isin( stock_code_letters) ]

# description
df4 = df4.drop( columns='description', axis=1 )

# map -  
df4 = df4[~df4['country'].isin( ['European Community', 'Unspecified' ] ) ]

# bad users
df4 = df4[~df4['customer_id'].isin( [16446] )]

# quantity
df_returns = df4.loc[df3['quantity'] < 0, :]
df_purchases = df4.loc[df3['quantity'] >= 0, :]

# 5.0 - FEATURE ENGINEERING

In [ ]:
df5 = df4.copy()

In [ ]:
df_reference = df5.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country'], axis = 1).drop_duplicates(ignore_index = True)

## 5.1 - GROSS REVENUE

In [ ]:
#gross Revenue (quantity * price)
df_purchase = df_purchases.copy()
df_purchase.loc[:, 'gross_revenue'] = df_purchase.loc[:, 'quantity'] * df_purchase.loc[:, 'unit_price']


#monetary
df_monetary = df_purchase.loc[:,['customer_id','gross_revenue']].groupby('customer_id').sum().reset_index()
df_reference = pd.merge(df_reference, df_monetary, on = 'customer_id', how = 'left')
df_reference.isna().sum()

## 5.2 - RECENCY - DAY FROM LAST PURCHASE

In [ ]:
#recency (Last Day Purchase)
df_recency = df_purchase.loc[:, ['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df5['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_reference = pd.merge( df_reference, df_recency, on='customer_id', how='left' )
df_reference.isna().sum()

## 5.3 - QUANTITY OF PURCHASED

In [ ]:
#frequency (Quantity of Purchase)
df_freq = (df_purchase.loc[:, ['customer_id', 'invoice_no']].drop_duplicates()
                                                             .groupby( 'customer_id' )
                                                             .count()
                                                             .reset_index()
                                                             .rename( columns={'invoice_no': 'qtde_invoices'}) )
df_reference = pd.merge( df_reference, df_freq, on='customer_id', how='left' )
df_reference.isna().sum()

## 5.4 - QUANTITY OF ITEMS PURCHASED

In [ ]:
#frequency (Quantity of Purchase)
df_freq = (df_purchase.loc[:, ['customer_id', 'quantity']].groupby( 'customer_id' ).sum()
                                                           .reset_index()
                                                           .rename( columns={'quantity': 'qtde_items'} ) )
df_reference = pd.merge( df_reference, df_freq, on='customer_id', how='left' )
df_reference.isna().sum()

## 5.5 - QUANTITY OF PRODUCTS PURCHASED

In [ ]:
df_freq = (df_purchase.loc[:, ['customer_id', 'stock_code']].groupby( 'customer_id' ).count()
                                                           .reset_index()
                                                           .rename( columns={'stock_code': 'qtde_products'} ) )
df_reference = pd.merge( df_reference, df_freq, on='customer_id', how='left' )
df_reference.isna().sum()

## 5.6 - AVERAGE TICKET VALUE

In [ ]:
#average ticket
df_avg_ticket = df_purchase.loc[:, ['customer_id', 'gross_revenue']].groupby( 'customer_id' ).mean().reset_index().rename( columns={'gross_revenue':'avg_ticket'} )
df_reference = pd.merge( df_reference, df_avg_ticket, on='customer_id', how='left')
df_reference.isna().sum()

## 5.7 - AVERAGE RECENCY DAYS

In [ ]:
df_aux = df4[['customer_id', 'invoice_date']].drop_duplicates().sort_values( ['customer_id', 'invoice_date'], ascending=False )
df_aux['next_customer_id'] = df_aux['customer_id'].shift() # next customer
df_aux['previous_date'] = df_aux['invoice_date'].shift() # next invoince date

df_aux['avg_recency_days'] = df_aux.apply( lambda x: ( x['invoice_date'] - x['previous_date'] ).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1 )

df_aux = df_aux.drop( ['invoice_date', 'next_customer_id', 'previous_date'], axis=1 ).dropna()

# average recency 
df_avg_recency_days = df_aux.groupby( 'customer_id' ).mean().reset_index()

# merge
df_reference = pd.merge( df_reference, df_avg_recency_days, on='customer_id', how='left' )
df_reference.isna().sum()

## 5.8 - FREQUENCY PURCHASE

In [ ]:
df_aux = ( df_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                             .groupby( 'customer_id')
                                                             .agg( max_ = ( 'invoice_date', 'max' ), 
                                                                   min_ = ( 'invoice_date', 'min' ),
                                                                   days_= ( 'invoice_date', lambda x: ( ( x.max() - x.min() ).days ) + 1 ),
                                                                   buy_ = ( 'invoice_no', 'count' ) ) ).reset_index()
# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if  x['days_'] != 0 else 0, axis=1 )

# Merge
df_reference = pd.merge( df_reference, df_aux[['customer_id', 'frequency']], on='customer_id', how='left' )

df_reference.isna().sum()

## 5.9 - NUMBER OF RETURNS

In [ ]:
df_returns = df_returns[['customer_id', 'quantity']].groupby( 'customer_id' ).sum().reset_index().rename( columns={'quantity':'qtde_returns'} )
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_reference = pd.merge( df_reference, df_returns, how='left', on='customer_id' )
df_reference.loc[df_reference['qtde_returns'].isna(), 'qtde_returns'] = 0

df_reference.isna().sum()

## 5.10 - BASKET SIZE

In [ ]:
df_aux = ( df_purchases.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby( 'customer_id' )
                                                                            .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                  n_products=( 'quantity', 'sum' ) )
                                                                            .reset_index() )

# calculation
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_reference = pd.merge( df_reference, df_aux[['customer_id', 'avg_basket_size']], how='left', on='customer_id' )
df_reference.isna().sum()

## 5.11 - UNIQUE ITENS BASKET SIZE

In [ ]:
df_aux = ( df_purchases.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby( 'customer_id' )
                                                                            .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                   n_products=( 'stock_code', 'nunique' ) )
                                                                            .reset_index() )

# calculation
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_reference = pd.merge( df_reference, df_aux[['customer_id', 'avg_unique_basket_size']], how='left', on='customer_id' )
df_reference.isna().sum()

## 5.12 - CLEANING NA

In [ ]:
df_reference = df_reference.dropna()
df_reference.isna().sum()

# 6.0 - EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
df6 = df_reference.copy()

## 6.2 - SPACE STUDY ANALYSIS

In [ ]:
df_space = df6.drop(columns = ['customer_id'], axis = 1).copy()
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtde_products', 'frequency', 'qtde_returns']
df_space = df6[cols_selected].copy()
df_space.columns

In [ ]:
mms = MinMaxScaler()

df_space['gross_revenue']               = mms.fit_transform(df_space[['gross_revenue']])
df_space['recency_days']                = mms.fit_transform(df_space[['recency_days']])
df_space['qtde_products']               = mms.fit_transform(df_space[['qtde_products']])
df_space['frequency']                   = mms.fit_transform(df_space[['frequency']])
df_space['qtde_returns']                = mms.fit_transform(df_space[['qtde_returns']])

### 6.2.1 - PCA ANALYSIS

In [ ]:
pca = PCA(n_components = df_space.shape[1])

principal_components = pca.fit_transform(df_space)

#plot explained variable
features = range(pca.n_components_)

plt.bar(features, pca.explained_variance_ratio_, color='darkgreen')
plt.title('PCA Space Analysis')
plt.xlabel('Data Variation X Axis')
plt.ylabel('Data Variation Y Axis')

for i, explained_var in enumerate(pca.explained_variance_ratio_):
    plt.text(i, explained_var + 0.005, f'{explained_var:.3f}', ha='center', va='bottom', fontsize=9)

plt.show()

In [ ]:
df_pca = pd.DataFrame(principal_components)
df_view = df_pca.loc[df_pca[0] <=3000].copy()
sns.scatterplot(data = df_view, x = 0, y = 1,)
plt.title('Distribuition Space Analysis')
plt.xlabel('Data Variation X Axis')
plt.ylabel('Data Variation Y Axis')
plt.show()

### 6.2.2 - UMAP ANALYSIS

In [ ]:
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_space)

#embedding
df_pca['embedding_x'] = embedding[:, 0]
df_pca['embedding_y'] = embedding[:, 1]

#plot

sns.scatterplot(data = df_pca, x = 'embedding_x', y = 'embedding_y')

plt.show()

### 6.2.3 - T-SNE ANALYSIS

In [ ]:
reducer = TSNE(random_state = 42, n_components = 2, n_jobs = -1, )
embedding = reducer.fit_transform(df_space)

#embedding
df_pca['embedding_x'] = embedding[:, 0]
df_pca['embedding_y'] = embedding[:, 1]

#plot

sns.scatterplot(data = df_pca, x = 'embedding_x', y = 'embedding_y')

plt.show()

### 6.2.4 - TREE-BASED EMBEDDING ANALYSIS

In [ ]:
#dataset split
x_train = df_space.drop(columns = ['customer_id', 'gross_revenue'], axis = 1)
y_train = df_space['gross_revenue']

#model definition
rf_model = RandomForestRegressor(n_estimators = 100, random_state = 42)

#model training
rf_model.fit(x_train, y_train)

#leaf
leaf = rf_model.apply(x_train)

#dataframe leaf
df_leaf = pd.DataFrame(leaf)
df_leaf.head()

In [ ]:
#Reduce of dimensionality
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_leaf)

#embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

#plot

sns.scatterplot(data = df_tree, x = 'embedding_x', y = 'embedding_y')

plt.show()

# 7.0 - DATA PREPARATION

In [ ]:
df7 = df_tree.copy()


## 7.1 - RESCALING

In [ ]:
#mms = MinMaxScaler()

#df7['gross_revenue']            = mms.fit_transform(df7[['gross_revenue']])
#df7['recency_days']             = mms.fit_transform(df7[['recency_days']])
#df7['quantity_stock_code']      = mms.fit_transform(df7[['quantity_stock_code']])
#df7['frequency']                = mms.fit_transform(df7[['frequency']])
#df7['returns']                  = mms.fit_transform(df7[['returns']])


# 8.0 - HYPERPARAMETER FINE-TUNNING

In [ ]:
df8 = df7.copy()

In [ ]:
#cols_selected = df8[['customer_id','gross_revenue', 'recency_days', 'quantity_stock_code', 'frequency', 'returns']]
#parameter_tuning = cols_selected.copy()
#parameter_tuning = parameter_tuning.drop(columns = ['customer_id'], axis = 1)
parameter_tuning = df8.copy()

In [ ]:
parameter_tuning

In [ ]:
cluster = np.arange(2, 26, 1)

## 8.1 - K-MEANS

In [ ]:
kmeans_list = []
for k in cluster:
    #model definition
    kmeans = KMeans(n_clusters = k)

    #model traning
    kmeans.fit(parameter_tuning)

    #validation
    labels = kmeans.predict(parameter_tuning)

    #performance
    sil = silhouette_score(parameter_tuning, labels, metric = 'euclidean')
    kmeans_list.append(sil)

In [ ]:
plt.plot(cluster, kmeans_list, linestyle = '--', marker = 'o', color = 'b')
plt.title('Silouette Score vs Kmeans')
plt.xlabel('K')
plt.ylabel('Silhouette Score')
plt.show() 


## 8.2 - GAUSSIAN MIXTURE MODEL (GMM)

In [ ]:
gmm_list = []
for k in cluster:
    #model definition
    gmm = GaussianMixture(n_components= k)

    #model traning
    gmm.fit(parameter_tuning)

    #validation
    labels = gmm.predict(parameter_tuning)

    #performance
    sil = silhouette_score(parameter_tuning, labels, metric = 'euclidean')
    gmm_list.append(sil)

In [ ]:
plt.plot(cluster, gmm_list, linestyle = '--', marker = 'o', color = 'b')
plt.title('Silouette Score vs Gaussian Mixture')
plt.xlabel('K')
plt.ylabel('Silhouette Score')
plt.show() 

## 8.3 - HIERARCHICAL CLUSTERING

In [ ]:
#Definition and training model
hc_model = linkage(parameter_tuning, 'ward')

In [ ]:
#plot
dendrogram(hc_model, leaf_rotation = 90, leaf_font_size = 8, truncate_mode='lastp', p = 12, show_contracted=True)
plt.show()


### 8.3.1 - HIERARCHICAL CLUSTERING SILHOUETTE SCORE

In [ ]:
hc_list = []
for k in cluster:
    #model definition
    hc_model = linkage(parameter_tuning, 'ward')

    #model predict
    labels = fcluster(hc_model, k, criterion = 'maxclust')

    #performance
    sil = silhouette_score(parameter_tuning, labels, metric = 'euclidean')

    hc_list.append(sil)

In [ ]:
plt.plot(cluster, hc_list, linestyle = '--', marker = 'o', color = 'b')
plt.title('Silouette Score vs Hierarchical Clustering')
plt.xlabel('K')
plt.ylabel('Silhouette Score')
plt.show() 

## 8.5 - RESULTS OF PERFORMANCE

In [ ]:
df_results = pd.DataFrame({'Kmeans': kmeans_list, 'Gaussian Mixture Model': gmm_list, 'Hierarchical Clustering': hc_list}).T
df_results.columns = cluster
df_results.style.highlight_max(color='darkgreen', axis = 1)

# 9.0 - MACHINE LEARNING MODEL

## 9.1 - K-MEANS

In [ ]:
#model definition
k = 8
kmeans = KMeans(init = 'random', n_clusters = k, n_init = 10, max_iter = 300, random_state = 42)

#model training
kmeans.fit(parameter_tuning)

#clustering
labels = kmeans.labels_

### 9.1.1 - CLUSTER PERFOMANCE

In [ ]:
print('WSS Value: {}'.format(kmeans.inertia_))
print('Silhouette Score Value: {}'.format(silhouette_score(parameter_tuning, labels, metric = 'euclidean')))

### 9.1.2 - CLUSTER ANALYSIS

In [ ]:
df10 = parameter_tuning.copy()
df10['cluster'] = labels
df10.head(10)

### 9.1.3 - VISUALIZATION INSPECTION

In [ ]:
plt.figure(figsize=(18, 11))
sns.scatterplot(data = df10, x = 'embedding_x', y = 'embedding_y', hue = 'cluster', palette='deep')
plt.show()

## 9.2 - GAUSSIAN MIXTURE MODEL

In [ ]:
k = 8
#model definition
gmm = GaussianMixture(n_components= k, random_state=42)

#model traning
gmm.fit(parameter_tuning)

#validation
labels = gmm.predict(parameter_tuning)

#performance
sil = silhouette_score(parameter_tuning, labels, metric = 'euclidean')
gmm_list.append(sil)

### 9.2.1 - CLUSTER PERFOMANCE

In [ ]:
print('Silhouette Score Value: {}'.format(silhouette_score(parameter_tuning, labels, metric = 'euclidean')))

### 9.2.2 - CLUSTER ANALYSIS

In [ ]:
df10 = parameter_tuning.copy()
df10['cluster'] = labels
df10.head(10)

### 9.2.3 - VISUALIZATION INSPECTION

In [ ]:
plt.figure(figsize=(18, 11))
sns.scatterplot(data = df10, x = 'embedding_x', y = 'embedding_y', hue = 'cluster', palette='deep')
plt.show()

# 10.0 - VISUALIZATION INSPECTION

## 10.1 - 10.1 - 2D PLOT

In [ ]:
df_viz = df10.copy()
sns.pairplot(df_viz, hue = 'cluster')
plt.show()

## 10.2 - UMAP - HIGH DIMENSIONALITY

In [ ]:
reducer = umap.UMAP(random_state = 42, n_neighbors = 200, n_jobs = 1)
embedding = reducer.fit_transform(parameter_tuning)

#embedding
df_viz['embedding_x'] = embedding[:, 0]
df_viz['embedding_y'] = embedding[:, 1]

#plot

sns.scatterplot(data = df_viz, x = 'embedding_x', y = 'embedding_y', 
                hue = 'cluster', 
                palette = sns.color_palette( 'hls', n_colors = len(df_viz['cluster'].unique())))

plt.show()

## 10.4 - CLUSTER PROFILE

In [ ]:
df_profile = df6[cols_selected].copy()
df_profile['cluster'] = labels

In [ ]:
df_profile['recency_days'] = df_profile['recency_days'].astype(int)
df_profile['qtde_products'] = df_profile['qtde_products'].astype(int)
df_profile['qtde_returns'] = df_profile['qtde_returns'].astype(int)

# Number of customer
df_cluster = df_profile[['customer_id', 'cluster']].groupby( 'cluster' ).count().reset_index()
df_cluster['percentual_customer'] = 100*( df_cluster['customer_id'] / df_cluster['customer_id'].sum() )

# Avg Gross revenue
df_avg_gross_revenue = df_profile[['gross_revenue', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_gross_revenue, how='inner', on='cluster' )

# Avg recency days
df_avg_recency_days = df_profile[['recency_days', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_recency_days, how='inner', on='cluster' )

# Avg invoice_no
df_qtde_products = df_profile[['qtde_products', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_products, how='inner', on='cluster' )

# Frequency
df_frequency = df_profile[['frequency', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_frequency, how='inner', on='cluster' )

# Returns
df_qtde_returns = df_profile[['qtde_returns', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_returns, how='inner', on='cluster' )

df_cluster['recency_days'] = df_cluster['recency_days'].astype(int)
df_cluster['qtde_products'] = df_cluster['qtde_products'].astype(int)
df_cluster['qtde_returns'] = df_cluster['qtde_returns'].astype(int)

df_cluster = df_cluster.sort_values('gross_revenue', ascending=False)

new_order_comuns = ['customer_id', 'cluster', 'percentual_customer', 'frequency','recency_days', 'qtde_products', 'qtde_returns', 'gross_revenue']
df_cluster = df_cluster[new_order_comuns]
df_cluster

 DIAMOND - CLUSTER 01 (Candidate High Value Customer):
 - Total of customers: 487 (16.03% of all customers)
 - Average of Gross Revenue: $9,544.57
 - Average of Recency Days: 20
 - Average of invoice number: 419
 - Average of Returns: 298
 - Average Purchase Frequency: 0.09%

 <br>

SILVER PRODUCTS - CLUSTER 05:
 - Total of customers: 523 (17.21% of all customers)
 - Average of Gross Revenue: $2,641.84
 - Average of Recency Days: 44
 - Average of invoice number: 140
 - Average of Returns: 24
 - Average Purchase Frequency: 0.10%

 <br>
 
BRONZE MONEY CLUSTER 07:
 - Total of customers: 417 (13.72% of all customers)
 - Average of Gross Revenue: $1,803.20
 - Average of Recency Days: 56
 - Average of invoice number: 88
 - Average of Returns: 10
 - Average Purchase Frequency: 0.10%

 <br>
 
BRONZE PRODUCTS CLUSTER 03:
 - Total of customers: 382 (12.57% of all customers)
 - Average of Gross Revenue: $1,198.02
 - Average of Recency Days: 61
 - Average of invoice number: 54
 - Average of Returns: 7
 - Average Purchase Frequency: 0.09%

 <br>
 
SILVER LESS DAYS - CLUSTER 02:
 - Total of customers: 343 (14.28% of all customers)
 - Average of Gross Revenue: $1,069.55
 - Average of Recency Days: 71
 - Average of invoice number: 45
 - Average of Returns: 6
 - Average Purchase Frequency: 0.09%

 <br>
 
SILVER LOWER PURCHASE - CLUSTER 0:
 - Total of customers: 168 (5.53% of all customers)
 - Average of Gross Revenue: $798.63
 - Average of Recency Days: 88
 - Average of invoice number: 27
 - Average of Returns: 4
 - Average Purchase Frequency: 0.17%

 <br>
 
SILVER STOP RETURNS - CLUSTER 06:
 - Total of customers: 435 (14.31% of all customers)
 - Average of Gross Revenue: $656.33
 - Average of Recency Days: 94	
 - Average of invoice number: 14
 - Average of Returns: 19
 - Average Purchase Frequency: 0.06%

 <br>
 
BRONZE LOWER PURCHASE - CLUSTER 04:
 - Total of customers: 193 (6.35% of all customers)
 - Average of Gross Revenue: $338.86
 - Average of Recency Days: 169	
 - Average of invoice number: 13
 - Average of Returns: 5
 - Average Purchase Frequency: 1.0%



# 11.0 - DEPLOY

## 11.1 - CREATE DB CONNECTION

In [ ]:
query_create_table_diamond = """
    CREATE TABLE diamond(
        customer_id INTEGER,
        gross_revenue REAL,
        recency_days INTEGER,
        qtde_products INTEGER,
        frequency REAL,
        qtde_returns INTEGER,
        cluster INTEGER
    )
"""

connection = sqlite3.connect('diamond_db.sqlite')
connection.execute(query_create_table_diamond)
connection.commit()
connection.close()


## 11.2 - INSERT DATA

In [ ]:
connection = create_engine('sqlite:///diamond_db.sqlite')
df_profile.to_sql('diamond', con = connection, if_exists='append', index = False)

## 11.3 - VIEW DATA

In [ ]:
query = """ SELECT * FROM diamond"""
df_query = pd.read_sql_query(query, connection)

In [ ]:
df_query